In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin
from pymysql import Connection
import time
import threading
import concurrent.futures
import re

In [3]:
url = 'https://www.biqg.cc/wangyou/'
r = requests.get(url)
html = r.text
print(html)

<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no">
<meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
<meta name="renderer" content="webkit|ie-comp|ie-stand">
<meta name="format-detection" content="telephone=no"/>
<title>网游竞技_小说更新列表_第1页_笔趣阁</title>
<meta name="keywords" content="网游竞技,笔趣阁" />
<meta name="description" content="笔趣阁是广大书友最值得收藏的网游竞技小说阅读网，分类书库收录了当前最好看的网游竞技小说，免费提供高质量的2022网游竞技小说更新列表_第1页，是广大爱好者必备的小说阅读网。" />
<link rel="stylesheet" href="/css/style.css" />
<script type="text/javascript" src="https://apps.bdimg.com/libs/jquery/1.8.3/jquery.min.js"></script>
<script type="text/javascript" src="/js/compc.js?v=1.23"></script>
</head>
<body>
<div class="header_wap pc_none">
	<a class="home" href="javascript:history.go(-1);"><svg class="lnr lnr-chevron-left-circle"><use xlink:href="#lnr-chevron-left-circle"></use></svg></a>
	<span class="title">网游竞

In [4]:
# 删除首页和排行榜，将数据标准化
soup = BeautifulSoup(html, 'html.parser')
li_tags = soup.find_all('li')
li_tags.pop(0) 
li_tags.pop(-1)

# 获取每个标签的href
href_list = []
for li_tag in li_tags:
    a_tag = li_tag.find('a')
    if a_tag:
        href_value = a_tag.get('href')
        href_list.append(href_value)
print(href_list)

# 获取每个标签的name
tag_list = []
for li_tag in li_tags:
    a_tag = li_tag.find('a')
    tag_list.append(a_tag.text)  
print(tag_list)

['/xuanhuan/', '/wuxia/', '/dushi/', '/lishi/', '/wangyou/', '/kehuan/', '/mm/', '/finish/']
['玄幻', '武侠', '都市', '历史', '网游', '科幻', '女生', '完本']


In [6]:
# 将获取到的href拼接起来，形成每个标签页面的url
category_list = []
for href in href_list:
    category_href = urljoin(url,href)
    category_list.append(category_href)
print(category_list)

['https://www.biqg.cc/xuanhuan/', 'https://www.biqg.cc/wuxia/', 'https://www.biqg.cc/dushi/', 'https://www.biqg.cc/lishi/', 'https://www.biqg.cc/wangyou/', 'https://www.biqg.cc/kehuan/', 'https://www.biqg.cc/mm/', 'https://www.biqg.cc/finish/']


In [9]:
# 获取到每个标签页面的每本书的url
def Get_book_url_list(category_list):
    book_list = []
    for url in category_list:
        # 找到每本书的div
        r = requests.get(url)
        html = r.text
        soup = BeautifulSoup(html, 'html.parser')
        book = soup.find_all('div', class_='image')
        book_3 = book[:3]
        for div in book_3:
            a_tag = div.find('a')
            href_value = a_tag.get('href')
            book_url = urljoin(url, href_value)
            book_list.append(book_url)
    return book_list

book_list = Get_book_url_list(category_list)
print(book_list[1])
print(book_list)

https://www.biqg.cc/book/134648/
['https://www.biqg.cc/book/143308/', 'https://www.biqg.cc/book/134648/', 'https://www.biqg.cc/book/148987/', 'https://www.biqg.cc/book/109312/', 'https://www.biqg.cc/book/3093/', 'https://www.biqg.cc/book/135908/', 'https://www.biqg.cc/book/147096/', 'https://www.biqg.cc/book/147189/', 'https://www.biqg.cc/book/162239/', 'https://www.biqg.cc/book/145331/', 'https://www.biqg.cc/book/155582/', 'https://www.biqg.cc/book/20043/', 'https://www.biqg.cc/book/161837/', 'https://www.biqg.cc/book/154736/', 'https://www.biqg.cc/book/140589/', 'https://www.biqg.cc/book/161187/', 'https://www.biqg.cc/book/102399/', 'https://www.biqg.cc/book/137851/', 'https://www.biqg.cc/book/150539/', 'https://www.biqg.cc/book/152542/', 'https://www.biqg.cc/book/161348/', 'https://www.biqg.cc/book/148812/', 'https://www.biqg.cc/book/147006/', 'https://www.biqg.cc/book/160900/']


In [ ]:
# 创建数据库连接
con = Connection(
    host="localhost",# 主机名
    port=3306, # 端口
    user="root", # 账户
    passwd="hk123" # 密码
)

# 设置游标
cursor = con.cursor()
# 选择数据库
cursor.execute("use library")
# 提交结果
con.commit()


# 执行查询
cursor.execute("SELECT DISTINCT book_name, title FROM book_content;")
# 获取结果
result = cursor.fetchall()
print(result)
book_name_list = []
for i in result:
    book_name_list.append(i[0])
print(book_name_list)


for book_url in book_list:
    r = requests.get(book_url)
    html = r.text
    soup = BeautifulSoup(html,'html.parser')

    # 找到标签信息
    tag_div = soup.find('div', class_='path')   # 找到包含标签的div
    tag = re.search(r'>\s*(.*?)\s*>', tag_div.text)
    tag = tag.group(1)

    # 找到书名
    h1 = soup.find('h1')

    # 找到作者名
    author_div = soup.find('div', class_='small')   # 找到包含作者信息的div标签
    author_span = author_div.select_one('div.small span:contains("作者")')  # 找到包含作者信息的<span>标签
    author = author_span.text.split('：')[1]    # 提取作者信息
    print(h1.text)

    # if h1.text not in book_name_list:
    #     insert_sql = "INSERT INTO books (book_name, author, tag) VALUES (%s, %s, %s)"
    #     cursor.execute(insert_sql, (h1.text, author, tag))
    #     con.commit() # 提交结果
    #     print(f"{h1.text}---{author}---{tag}---写入成功！")
    print("=================")


con.close()